# 建模流程Check list

## 问题定义（打标）

1. 分析业务流程并挑选关键环节作为切入点，挑选原则
   - 具有业务意义，*如：KPI指标、或转化较低的环节等*
   - 有可靠数据支持， *如：无法获取客户浏览短信的数据，则不能将浏览与否作为打标依据，可考虑使用登录行为替代*

2. 将业务目标抽象为具体的问题
   - 分类
   - 聚类
   - 回归等

## SQL（数据粗加工）

1. 样本选取
   - 根据业务目标确定目标客群
   - 根据目标客群数据量判断是否需要抽样


2. 特征粗筛
   - 为避免数据量过大，需根据业务目标挑选相关分类的特征进行加工

> 注意：观察期（特征加工）与表现期（打标）使用的数据不能有时间上的重叠

3. **Code check list**
   - 没有按照通用的格式对齐各字段，字段类型以及字段说明等
   - 没有在`case when`语句最后添加上`else null end`或者`else 0 end`
   - 在impala查询语句中，不能在`left join`条件里面加上不等式，如：>=，<=等
   - sql语句中存在大量不必要的`select distinct`语句
   - 存在类似于`select *`，`select count(*)`的语句，不符合规范，尽量只选取需要的列
   - 注意少用子查询，可用`with subquerry as table name`语法代替提高并行度，对于反复使用的逻辑建立中间表提高代码复用率
   - 注意使用**谓词下推**提高运行效率，将过滤表达式尽可能移动至靠近数据源的位置，以使真正执行时能直接跳过无关的数据

## Python（数据细加工&建模）

1. 数据探索
   - 特征中能否挖掘出有效的专家规则？
      - 若有则可以直接通过部署专家规则的形式解决问题，节省开发时间
      - 方法：绘制分布图、相关性分析等
   - 样本是否平衡？
      - 若极不平衡(通常正负样本比例<1/10)，则需进行样本平衡
      - 方法：过（欠）采样、人工合成少数样本、迁移学习等

2. 特征加工（根据业务、数据以及所选模型情况决定）
   - 数据类型转化
   - 空缺值&异常值处理
   - 标准化、归一化（针对特定模型，如：广义线性模型）
   - 分桶
   - 特征交叉
   - 降维

3. 训练、验证和测试样本划分
   - 训练：在部分样本中学习数据的概率分布
   - 验证：用训练数据之外的数据调整模型超参，平衡偏差和方差，提高模型泛化性能
   - 测试：用训练及验证之外的数据对模型性能进行离线测试

> 注意：
> 1. 训练、验证和测试数据集尽量不要产生人与时间的交叉 
> 2. 为避免由于人为数据集划分不均导致的测试效果不客观，可以使用交叉验证

4. 模型训练&评估
   - Baseline model
   - 树模型
   - 集成模型
   - 神经网络

5. 效果&效能提升
   - 特征选择：根据模型初步训练结果筛选重要特征
   - 模型选择：根据模型初步训练结果筛选效果好的模型
   - 模型融合：将多个模型结果进行融合
   - Refit：将验证与测试数据集用于训练

> 注意：模型融合中子模型最好分属于不同分类

6. Code check list
   - 代码注释是否清晰，命名是否规范
      - 变量&函数命名:小写字母，单词之间用下划线分割，如：`local_var, get_value()`
      - 模块&包命名：小写字母，单词之间用下划线分割，如：`tree_stats.py`
      - 类命名：单词首字母大写，如：`TreeNode`
   - 变量统一定义，是否存在硬编码现象
   - 重复的操作是否抽象成模块、类或函数
   - 循环操作是否可用矩阵操作替代

7. 例子
   - 函数注释
   ```
   def function(a, b):
    """Do X and return a list
    
    
    """
    return
   ```

    def function(a, b):
    """Do X and return a list
    
    
    """
    return

In [ ]:
def function(a, b):
    """Add a, b and return the result
    
    
    """
    return a+b